# 4_Use_TensorBoard_to_Understand_Debug_and_optimize_TensorFlow_programs

我们集成了名为 TensorBoard 的可视化工具，来帮助您理解，调试和优化您的TensorFlow程序。 
您可以使用TensorBoard来可视化您的TensorFlow图形，绘制关于图形执行的量化指标，并显示其他数据。
下面是一个MNIST的小例子，来帮助您了解并使用 TensorBoard。

*Important: 请确保将您的logs文件保存在 `./logs`文件夹下*

In [1]:
from __future__ import print_function

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1
# ！Important！
# *您需要将logs文件存储到`/logs`文件夹下*
logs_path = './logs/tutorial4/'

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))


Extracting ./tmp/data/train-images-idx3-ubyte.gz
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost= 1.183653277
Epoch: 0002 cost= 0.665266814
Epoch: 0003 cost= 0.552696104
Epoch: 0004 cost= 0.498522807
Epoch: 0005 cost= 0.465410931
Epoch: 0006 cost= 0.442538291
Epoch: 0007 cost= 0.425468006
Epoch: 0008 cost= 0.412111081
Epoch: 0009 cost= 0.401391366
Epoch: 0010 cost= 0.392365176
Epoch: 0011 cost= 0.384755743
Epoch: 0012 cost= 0.378185968
Epoch: 0013 cost= 0.372429544
Epoch: 0014 cost= 0.367311060
Epoch: 0015 cost= 0.362732099
Epoch: 0016 cost= 0.358580897
Epoch: 0017 cost= 0.354855532
Epoch: 0018 cost= 0.351457369
Epoch: 0019 cost= 0.348307079
Epoch: 0020 cost= 0.345436401
Epoch: 0021 cost= 0.342750962
Epoch: 0022 cost= 0.340235087
Epoch: 0023 cost= 0.337916244
Epoch: 0024 cost= 0.335736679
Epoch: 0025 cost= 0.333718029
Optimization Finished!
Accuracy: 0.9137


## 查看您的模型结构和训练情况
现在，来查看您的模型的结构和训练情况吧。

<img src="./media/tensorboard.gif" width="70%" height="70%" />  


更多关于TensorBoard的信息请参考 https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard 和
https://github.com/tensorflow/tensorboard